<a href="https://colab.research.google.com/github/victoregomezf-maker/seminario/blob/main/Predicci%C3%B3n_de_demanda_insumos_medicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicción de demanda insumos medicos**

In [ ]:
import pandas as pd

# 1. CARGAR EL ARCHIVO TOTALMENTE LIMPIO
ruta = "/content/DEMANDA AÑO 2025.csv"
df_raw = pd.read_csv(ruta, sep=';', header=None, encoding='latin-1')

resultados = []

i = 0
while i < len(df_raw):
    fila_actual = df_raw.iloc[i].astype(str).tolist()
    fila_texto = " ".join(fila_actual).upper()

    # Detectamos el encabezado del mes (CANTIDAD DIARIA MES...)
    if "CANTIDAD DIARIA MES" in fila_texto:
        # Extraemos el mes después del guion o espacio
        nombre_mes = fila_texto.split("MES")[-1].replace("DE", "").strip("- ").strip()

        # La fila de los números (1, 2, 3...) está justo abajo
        fila_dias = [str(x).strip().replace('.0', '') for x in df_raw.iloc[i+1]]

        # Buscamos dónde empieza el día "1"
        try:
            col_inicio_datos = fila_dias.index('1')
        except ValueError:
            i += 1
            continue

        # Bajamos a los productos
        j = i + 2
        while j < len(df_raw):
            datos_fila = df_raw.iloc[j].tolist()
            producto = str(datos_fila[0]).strip()

            # Si la fila está vacía o empieza otro mes, saltamos al siguiente bloque
            if producto.lower() in ['nan', ''] or "CANTIDAD DIARIA" in producto.upper():
                break

            # Extraemos la demanda para cada día numerado
            for idx_col in range(col_inicio_datos, len(datos_fila)):
                dia_label = fila_dias[idx_col]

                if dia_label.isdigit():
                    valor_celda = str(datos_fila[idx_col]).replace(',', '.').strip()
                    try:
                        # Si la celda está vacía (;;), float() fallará y pondrá 0.0
                        cantidad = float(valor_celda)
                    except:
                        cantidad = 0.0

                    resultados.append({
                        "Producto": producto,
                        "Mes": nombre_mes,
                        "Dia": int(dia_label),
                        "Cantidad": cantidad
                    })
            j += 1
        i = j
    else:
        i += 1

# 3. CREAR TABLA FINAL
df_final = pd.DataFrame(resultados)

if not df_final.empty:
    print(f"✅ ¡Análisis exitoso!")
    print(f"Se detectaron productos como: {df_final['Producto'].iloc[0]}")
    # Mostramos los datos donde hubo consumo real
    df_con_datos = df_final[df_final['Cantidad'] > 0]
    display(df_con_datos.head(20))
else:
    print("❌ No se encontraron datos. Verifica que el archivo use ';' como separador.")

✅ ¡Análisis exitoso!
Se detectaron productos como: 00031P - ELECTRODO ADULTO REF. 0651 30 77007 DESFIBRILADOR D1 D3 D6 MINDRAY MR60


,Producto,Mes,Dia,Cantidad
0,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,1,1.0
2,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,3,1.0
3,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,4,1.0
5,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,6,1.0
6,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,7,1.0
7,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,8,2.0
9,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,10,2.0
12,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,13,2.0
13,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,14,1.0
16,00031P - ELECTRODO ADULTO REF. 0651 30 77007 D...,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,17,1.0


In [ ]:
import pandas as pd

# 1. Verificar cuántos meses únicos se detectaron
meses_detectados = df_final['Mes'].unique()
print(f"--- REVISIÓN DE MESES ---")
print(f"Meses encontrados ({len(meses_detectados)}): {meses_detectados}")

# 2. Resumen por mes: Conteo de productos y suma de cantidades
resumen = df_final.groupby('Mes').agg(
    Total_Productos=('Producto', 'nunique'),
    Registros_Dias=('Cantidad', 'count'),
    Suma_Total_Demanda=('Cantidad', 'sum')
).reset_index()

print("\n--- RESUMEN DE DATOS POR MES ---")
display(resumen)

# 3. Verificar si hay productos que quedaron en cero (posibles errores de lectura)
productos_con_datos = df_final[df_final['Cantidad'] > 0]['Producto'].nunique()
total_productos = df_final['Producto'].nunique()

print("\n--- CONSISTENCIA DE DATOS ---")
print(f"Productos totales en el archivo: {total_productos}")
print(f"Productos con al menos una demanda registrada: {productos_con_datos}")

if len(meses_detectados) < 10:
    print("\n⚠️ ALERTA: No se detectaron los 10 meses. Revisa si los títulos 'CANTIDAD DIARIA MES' están escritos igual en todo el archivo.")
else:
    print("\n✅ Los 10 meses parecen estar presentes.")

--- REVISIÓN DE MESES ---
Meses encontrados (10): ['FEBRERO  NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN'
 'MARZO  NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN'
 'ABRIL  NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN'
 'MAYO  NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN'
 'JUNIO  NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN'
 'JULIO  NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN'
 'AGOSTO  NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN'
 'SEPTIEMBRE  NAN NAN NAN NAN NAN NAN 

,Mes,Total_Productos,Registros_Dias,Suma_Total_Demanda
0,ABRIL NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN...,207,6210,325347.0
1,AGOSTO NAN NAN NAN NAN NAN NAN NAN NAN NAN NA...,176,5456,189158.0
2,FEBRERO NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,154,4340,241681.0
3,JULIO NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN...,214,6634,343257.0
4,JUNIO NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN...,202,6060,407528.0
5,MARZO NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN...,174,5394,331286.0
6,MAYO NAN NAN NAN NAN NAN NAN NAN NAN NAN NAN ...,181,5642,406166.0
7,NOVIEMBRE NAN NAN NAN NAN NAN NAN NAN NAN NAN...,232,6960,440611.0
8,OCTUBRE NAN NAN NAN NAN NAN NAN NAN NAN NAN N...,209,6510,387005.0
9,SEPTIEMBRE NAN NAN NAN NAN NAN NAN NAN NAN NA...,248,7440,383691.0



--- CONSISTENCIA DE DATOS ---
Productos totales en el archivo: 435
Productos con al menos una demanda registrada: 432

✅ Los 10 meses parecen estar presentes.
